# Eventhandling in Input Connectors get_next() - internal event_backlog handling

A small demonstration of the internal `event_backlog` of the Input Connecor and the `acknowledge(...)`  method.

1. Prepare the kafka

In [1]:
%%bash
# starting kafka container
docker compose -f ../../../../../examples/compose/docker-compose.yml down -v
docker compose -f ../../../../../examples/compose/docker-compose.yml up -d kafka
# creating the topic
docker exec -i kafka /bin/bash -c "/opt/bitnami/kafka/bin/kafka-topics.sh --create --topic consumer --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1"
# waiting for kafka to be ready
sleep 10


bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
 Container kafka  Stopping
 Container kafka  Stopped
 Container kafka  Removing
 Container kafka  Removed
 Network compose_kafka  Removing
 Network compose_kafka  Removed
 Network compose_kafka  Creating
 Network compose_kafka  Created
 Container kafka  Creating
 Container kafka  Created
 Container kafka  Starting
 Container kafka  Started
[2025-08-25 08:44:20,589] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Node may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-08-25 08:44:20,694] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Node may not be available. (org.apache.kafka.clients.NetworkClient)
[2025-08-25 08:44:20,795] WARN [AdminClient clientId=adminclient-1] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Node may not be availabl

Created topic consumer.


2. produce messages

In [2]:
%%bash
# producing 3 events to kafka topic consumer
docker exec -i kafka bash -lc '
printf "%s\n" \
  "{\"message\":\"the message 1\"}" \
  "{\"message\":\"the message 2\"}" \
  "{\"message\":\"the message 3\"}" \
| /opt/bitnami/kafka/bin/kafka-console-producer.sh --bootstrap-server localhost:9092 --topic consumer
'

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


3. initializing the confluentkafka_input connector

In [3]:
import sys
import logging
from logprep.factory import Factory
from logprep.ng.connector.confluent_kafka.input import ConfluentKafkaInput

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    stream=sys.stdout
)

# Create a Kafka input connector configuration

kafka_config = {
    "kafka": {
        "type": "ng_confluentkafka_input",
        "topic": "consumer",
        "kafka_config": {
            "bootstrap.servers": "127.0.0.1:9092",
            "group.id": "cgroup3",
            "enable.auto.commit": "true",
            "auto.commit.interval.ms": "10000",
            "enable.auto.offset.store": "false",
            "queued.min.messages": "100000",
            "queued.max.messages.kbytes": "65536",
            "statistics.interval.ms": "60000"
        }
    }
}

kafka_connector: ConfluentKafkaInput = Factory.create(kafka_config)

# Start the connector
kafka_connector.setup()

# show the current backlog
print(f"{kafka_connector.event_backlog.backlog=}")

DEBUG:Component:Checking health of kafka
kafka_connector.event_backlog.backlog=set()


4. Consume and simulate event processing

In [4]:
from logprep.ng.event.event_state import EventStateType

# Consume messages from the Kafka topic
event_1 = next(kafka_connector(timeout=10))
# - Event backlog size: 1
print(
    "\nExpected: 1 Event in Event-Backlog:"
    f"\n{kafka_connector.event_backlog.backlog=}"
    f"\nSize: {len(kafka_connector.event_backlog.backlog)}",
    end="\n\n"
)

event_2 = next(kafka_connector(timeout=10))
# - Event backlog size: 2 (
print(
    "Expected: 2 Events in Event-Backlog:"
    f"\n{kafka_connector.event_backlog.backlog=}"
    f"\nSize: {len(kafka_connector.event_backlog.backlog)}",
    end="\n\n"
)


# Fake/simulate processing:
# - set event directly to ACKED (will be removed in next get cycle)

print("... simulate event processing: set Event 1 to ACKED", end="\n\n")
event_1.state.current_state= EventStateType.ACKED

print("... simulate event processing: set Event 2 to DELIVERED", end="\n\n")
# - set event directly to DELIVERED (will be acked in next get cycle)
event_2.state.current_state= EventStateType.DELIVERED


# next call should ack all events with state DELIVERED and remove ACKED events from event_backlog
event_3 = next(kafka_connector(timeout=10))
# - Event backlog size: 1 (
print(
    "Expected: 2 Events in Event-Backlog:\n"
    "- 1. Event 1 (ACKED) removed from backlog\n"
    "- 2. Event 2 (DELIVERED) switch to ACKED\n"
    f"\n{kafka_connector.event_backlog.backlog=}"
    f"\nSize: {len(kafka_connector.event_backlog.backlog)}",
    end="\n\n"
)

# Shut down the connector to close the producer
kafka_connector.shut_down()

INFO:KafkaInput:bs0010tnu-devclient-PipelineNone-pid41021-a1f9f819-f067-4867-b099-a3ff1267a944 was assigned to topic: consumer | partition 0

Expected: 1 Event in Event-Backlog:
kafka_connector.event_backlog.backlog={LogEvent(data={'message': 'the message 1'}, state=received)}
Size: 1

Expected: 2 Events in Event-Backlog:
kafka_connector.event_backlog.backlog={LogEvent(data={'message': 'the message 1'}, state=received), LogEvent(data={'message': 'the message 2'}, state=received)}
Size: 2

... simutate event porcessing: set Event 1 to ACKED

... simutate event porcessing: set Event 2 to DELIVERED

Expected: 2 Events in Event-Backlog:
- 1. Event 1 (ACKED) removed from backlog
- 2. Event 2 (DELVIVERED) switch to ACKED

kafka_connector.event_backlog.backlog={LogEvent(data={'message': 'the message 2'}, state=acked), LogEvent(data={'message': 'the message 3'}, state=received)}
Size: 2

